In [1]:
import requests
import time
import numpy as np
import uuid
import shutil
import yaml

Define some useful functions and the addresses of the bServer and mServer globally

In [2]:
bServer = "http://127.0.0.1:18085/SIS/"
mServer = "http://127.0.0.1:18080"

In [3]:
def create_UUID():
    """UUIDs will be unique for a given run of the kernel"""
    unique_uuid = uuid.uuid4().hex[:8].upper()
    return unique_uuid


def create_SPECnewfile(new_name):
    t0=time.time()
    
    wait_until_SPECfinished()
    
    payload = {'spec_cmd': 'newfile {}'.format(new_name)}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent

def cam_refresh():
    t0=time.time()
    
    wait_until_SPECfinished()
    
    payload = {'spec_cmd': 'camrefresh'}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    time.sleep(45)
    wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent


def take_pilatus_image(exp_time, move_image_to=None):
    wait_until_SPECfinished()
    
    
    payload = {'spec_cmd': 'loopscan 1 {}'.format(exp_time)}
    r = requests.get(bServer + "execute_command", params=payload)
    
    wait_until_SPECfinished()

    counters = read_counters()
    
    image_savedpath = None
    if move_image_to is not None:
        pass
        
    
    return counters

def move_SPECmotor(motor_name, value, wait_until_finished = True):
    t0=time.time()
    payload = {'spec_cmd': 'mv {} {}'.format(motor_name, value)}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    if wait_until_finished is True:
        wait_until_SPECfinished()

    time_spent = time.time() - t0
    return time_spent

def set_SPECshutter(shutterOpen):
    t0=time.time()
    if shutterOpen is True:
        payload = {'spec_cmd': 'sopen'}
    else:
        payload = {'spec_cmd': 'sclose'}
    
    r = requests.get(bServer + "execute_command", params=payload)
    #print(r)
    #print("Response: ", r.json()['data'])
    
    time.sleep(0.1)
    
    time_spent = time.time() - t0
    return time_spent

def wait_until_SPECfinished():
    t0 = time.time()
    while True:
        r = requests.get(bServer + "is_busy")
        is_busy = r.json()['data']
            
        if is_busy is False:
            break
    return time.time() - t0

def read_counters(save_to_file=None):
    t0 = time.time()
    
    #Get the mnemonics - a little wasteful but that is ok I think right now.
    r = requests.get(bServer + "get_all_counter_mnemonics", params={})
    counter_mnemonics = r.json()['data']

    wait_until_SPECfinished()
    
    
    r = requests.get(bServer + "get_all_counters", params={})
    counter_values = r.json()['data']
   
    time_spent = time.time() - t0
    
    return dict(zip(counter_mnemonics, counter_values))


def move_to_ROSSindex(ross_index):
    r = requests.post("http://127.0.0.1:18080/combined_move/move_to_index/{}".format(ross_index))
    #print(r)
    #print(r.json())

In [4]:
g = read_counters()
print(g)

{'sec': 5.0, 'mon': 0.29173, 'det': 0.0, 'dead': 0.0, 'i1': 0.00223, 'i2': 0.04769, 'i3': 0.0, 'i4': 0.0, 'normlz': 0.0, 'fstat': 0.0, 'TEMP': 0.0, 'pd1': 42889691.0, 'pd2': 133.0, 'pd3': 6985.0, 'pd4': 59.0, 'pd5': 59.0, 'pd6': 59.0, 'pd7': 59.0, 'pd8': 59.0, 'pd9': 59.0, 'pd10': 59.0, 'pd11': 59.0, 'pd12': 59.0, 'pd13': 59.0, 'pd14': 59.0, 'pd15': 59.0, 'pd16': 59.0, 'ICR': 0.0, 'OCR': 0.0, 'ROI1': 0.0, 'ROI2': 0.0, 'ROI3': 0.0, 'ROI4': 0.0, 'keith_I': 0.0, 'keith_V': 0.0, 'CTEMP': 0.0, 'timesec': 0.0, 'tempsec': 0.0, 'PD_MIN': 0.0, 'PD_MAX': 0.0, 'A34410': 0.0, 'agdaq1': 0.0, 'agdaq2': 0.0, 'agdaq3': 0.0, 'agdaq4': 0.0, 'CTEMPB': 0.0}


In [16]:
cam_refresh()

45.07417416572571

In [5]:
move_SPECmotor('spr3', 51.1)

0.637932538986206

In [7]:
move_to_ROSSindex(2)

# Lets do some real work.... starting with the parameters defining the run.

Define the run name

In [5]:
scan_base_name = 'insitu_C2_fourthPressureSet'
path_to_detector = ""
path_to_saved_data = ""



Define the sample locations

In [7]:
sample_motors = {'x': 'spr2', 'y': 'spr3'}

sample_ROSSindex = {'QR1': "1", 'QR3': "0", 'air': "2"}
#Offset positions 
sample_global_offsets = {'QR1': {'x':47, 'y':50.75}, 'QR3': {'x':47, 'y':51.75}, 'air': {'x':49, 'y':52.75}} #Pressure set 4

#sample_global_offsets = {'QR1': {'x':47, 'y':50.50}, 'QR3': {'x':47, 'y':51.50}, 'air': {'x':49, 'y':52.50}} #Pressure set 3
#sample_global_offsets = {'QR1': {'x':47, 'y':50.25}, 'QR3': {'x':47, 'y':51.25}, 'air': {'x':49, 'y':52.25}} #Pressure set 2
#sample_global_offsets = {'QR1': {'x':47, 'y':50}, 'QR3': {'x':47, 'y':51}, 'air': {'x':49, 'y':52}}


exposure_time = 5
cam_refresh_freq = 10 #Number of scans before calling a cam refresh

Generate the positions to scan for each sample

In [8]:
sample_positions = {'QR1': [], 'QR3': [], 'air': []}


for rel_x_pos in np.linspace(0, 5, 5):
    for rel_y_pos in np.linspace(0, 5, 5):
        sample_positions['QR1'].append({'x': rel_x_pos + sample_global_offsets['QR1']['x'], 'y': rel_y_pos + sample_global_offsets['QR1']['y']})
        sample_positions['QR3'].append({'x': rel_x_pos + sample_global_offsets['QR3']['x'], 'y': rel_y_pos + + sample_global_offsets['QR3']['y']})
        
sample_positions['air'].append({'x': 0 + sample_global_offsets['air']['x'], 'y': 0 + sample_global_offsets['air']['y']})


#Rebuild the sample_scan_list. This is a bit more flexible way of constructing scans (e.g., you can interleave the samples or scan over positions in the sample... up to you)

sample_scan_list = []
sample_scan_list.append( {'QR1': sample_positions['QR1']})
sample_scan_list.append( {'QR3': sample_positions['QR3']})
sample_scan_list.append( {'air': sample_positions['air']})

print(sample_scan_list)

[{'QR1': [{'x': 47.0, 'y': 50.75}, {'x': 47.0, 'y': 52.0}, {'x': 47.0, 'y': 53.25}, {'x': 47.0, 'y': 54.5}, {'x': 47.0, 'y': 55.75}, {'x': 48.25, 'y': 50.75}, {'x': 48.25, 'y': 52.0}, {'x': 48.25, 'y': 53.25}, {'x': 48.25, 'y': 54.5}, {'x': 48.25, 'y': 55.75}, {'x': 49.5, 'y': 50.75}, {'x': 49.5, 'y': 52.0}, {'x': 49.5, 'y': 53.25}, {'x': 49.5, 'y': 54.5}, {'x': 49.5, 'y': 55.75}, {'x': 50.75, 'y': 50.75}, {'x': 50.75, 'y': 52.0}, {'x': 50.75, 'y': 53.25}, {'x': 50.75, 'y': 54.5}, {'x': 50.75, 'y': 55.75}, {'x': 52.0, 'y': 50.75}, {'x': 52.0, 'y': 52.0}, {'x': 52.0, 'y': 53.25}, {'x': 52.0, 'y': 54.5}, {'x': 52.0, 'y': 55.75}]}, {'QR3': [{'x': 47.0, 'y': 51.75}, {'x': 47.0, 'y': 53.0}, {'x': 47.0, 'y': 54.25}, {'x': 47.0, 'y': 55.5}, {'x': 47.0, 'y': 56.75}, {'x': 48.25, 'y': 51.75}, {'x': 48.25, 'y': 53.0}, {'x': 48.25, 'y': 54.25}, {'x': 48.25, 'y': 55.5}, {'x': 48.25, 'y': 56.75}, {'x': 49.5, 'y': 51.75}, {'x': 49.5, 'y': 53.0}, {'x': 49.5, 'y': 54.25}, {'x': 49.5, 'y': 55.5}, {'x':

Here we interleave some of the measurements between the two batteries to improve time resolution. (Not using this)

sample_positions = {'QR1': [], 'QR3': [], 'air': []}

sample_scan_list = []

for rel_x_pos in np.linspace(0, 5, 5):
    
    sample_positions['QR1'] = []
    sample_positions['QR3'] = []
    
    for rel_y_pos in np.linspace(0, 5, 10):
        sample_positions['QR1'].append({'x': rel_x_pos + sample_global_offsets['QR1']['x'], 'y': rel_y_pos + sample_global_offsets['QR1']['y']})
        sample_positions['QR3'].append({'x': rel_x_pos + sample_global_offsets['QR3']['x'], 'y': rel_y_pos + + sample_global_offsets['QR3']['y']})
        
    sample_scan_list.append( {'QR1': sample_positions['QR1']})
    sample_scan_list.append( {'QR3': sample_positions['QR3']})
    

sample_positions['air'].append({'x': 0 + sample_global_offsets['air']['x'], 'y': 0 + sample_global_offsets['air']['y']})

sample_scan_list.append( {'air': sample_positions['air']})

print(sample_scan_list)

Take control of SPEC

In [17]:
r = requests.get(bServer + "get_remote_control")
print(r.json())

{'help': 'Get the current plot. Not sure what this actually is', 'data': 'client in control.'}


## Ok. Lets do the actual scanning

In [9]:
#These previous values are used to prevent unnecessary motor movements in long scans. Set to None in the beginning to force the first move.
prev_ROSSindex = None
prev_x_pos = None
prev_y_pos = None

ordering_index = 0
do_cam_refresh = True
for scan_ctr in range(20000): #Set number scans to something large?
    tscan = time.time()
    
    #Check if we need to do a cam refresh
    if do_cam_refresh is True:
        if scan_ctr % cam_refresh_freq == 0: 
            print("Doing a cam refresh")
            cam_refresh()
    else:
        print("RE ENABLE CAM REFRESH checking!!!!!")    
    
    #Loop through each elemnt in the sample scan_list
    for single_scan_list in sample_scan_list:
        
        #Extract the sample name and the positions over which to scan
        for (sample_name, sample_pos_list) in single_scan_list.items():
            

            #Move to the right position on the ROSS switcher
            new_ROSSindex = sample_ROSSindex[sample_name]
            if prev_ROSSindex != new_ROSSindex:
                move_to_ROSSindex(new_ROSSindex)
                prev_ROSSindex = new_ROSSindex

            #Make a new SPEC file for each scan
            scan_SPECfilename = "{}_i{}_loop{}_{}".format(scan_base_name, ordering_index, scan_ctr, sample_name)
            ordering_index = ordering_index + 1
            print("Creating new SPEC file: {}".format(scan_SPECfilename))
            create_SPECnewfile(scan_SPECfilename)

            
            #Loop through the positions on the sample and take data
            pos_ctr = 0
            for xy_pos in sample_pos_list:

                #Compute the coordinates
                x_pos = xy_pos['x']
                y_pos = xy_pos['y']

                #Move the SPEC motors to the correct positions
                tSPECmove = time.time()
                
                #Check if we actually need to move x
                if prev_x_pos != x_pos:
                    move_SPECmotor(sample_motors['x'], x_pos)
                    prev_x_pos = x_pos

                #Check if we actually need to move y
                if prev_y_pos != y_pos:
                    move_SPECmotor(sample_motors['y'], y_pos)
                    prev_y_pos = y_pos

                print("{} pos {} ({},{}): move took {}s".format(sample_name, pos_ctr, x_pos, y_pos, time.time() - tSPECmove))



                #Take data
                f_name = "{}_{}_p{}_sc{}".format(scan_base_name, sample_name, pos_ctr, scan_ctr)
                take_pilatus_image(exposure_time)

                

                pos_ctr = pos_ctr + 1 #Increment the position counter

    print("Scan {} finished in {}s".format(scan_ctr, time.time() - tscan))
    #time.sleep(240.0) #wait?


            
        
        
        
        

Doing a cam refresh
Creating new SPEC file: insitu_C2_fourthPressureSet_i0_loop0_QR1
QR1 pos 0 (47.0,50.75): move took 0.786160945892334s
QR1 pos 1 (47.0,52.0): move took 0.3522303104400635s
QR1 pos 2 (47.0,53.25): move took 0.3505232334136963s
QR1 pos 3 (47.0,54.5): move took 0.35367441177368164s
QR1 pos 4 (47.0,55.75): move took 0.3499796390533447s
QR1 pos 5 (48.25,50.75): move took 1.149519920349121s
QR1 pos 6 (48.25,52.0): move took 0.3499789237976074s
QR1 pos 7 (48.25,53.25): move took 0.3501007556915283s
QR1 pos 8 (48.25,54.5): move took 0.36026668548583984s
QR1 pos 9 (48.25,55.75): move took 0.3499612808227539s
QR1 pos 10 (49.5,50.75): move took 1.1572396755218506s
QR1 pos 11 (49.5,52.0): move took 0.35161256790161133s
QR1 pos 12 (49.5,53.25): move took 0.35117602348327637s
QR1 pos 13 (49.5,54.5): move took 0.34670114517211914s
QR1 pos 14 (49.5,55.75): move took 0.3544425964355469s
QR1 pos 15 (50.75,50.75): move took 1.150381088256836s
QR1 pos 16 (50.75,52.0): move took 0.354255

QR3 pos 12 (49.5,54.25): move took 0.34999632835388184s
QR3 pos 13 (49.5,55.5): move took 0.3399631977081299s
QR3 pos 14 (49.5,56.75): move took 0.34997987747192383s
QR3 pos 15 (50.75,51.75): move took 1.1407508850097656s
QR3 pos 16 (50.75,53.0): move took 0.3428206443786621s
QR3 pos 17 (50.75,54.25): move took 0.34779953956604004s
QR3 pos 18 (50.75,55.5): move took 0.3408229351043701s
QR3 pos 19 (50.75,56.75): move took 0.340803861618042s
QR3 pos 20 (52.0,51.75): move took 1.1444165706634521s
QR3 pos 21 (52.0,53.0): move took 0.33982062339782715s
QR3 pos 22 (52.0,54.25): move took 0.3699779510498047s
QR3 pos 23 (52.0,55.5): move took 0.34999728202819824s
QR3 pos 24 (52.0,56.75): move took 0.3499789237976074s
Creating new SPEC file: insitu_C2_fourthPressureSet_i8_loop2_air
air pos 0 (49,52.75): move took 1.1975650787353516s
Scan 2 finished in 318.7943685054779s
Creating new SPEC file: insitu_C2_fourthPressureSet_i9_loop3_QR1
QR1 pos 0 (47.0,50.75): move took 0.7940340042114258s
QR1 pos

QR1 pos 22 (52.0,53.25): move took 0.3399791717529297s
QR1 pos 23 (52.0,54.5): move took 0.34999608993530273s
QR1 pos 24 (52.0,55.75): move took 0.3499782085418701s
Creating new SPEC file: insitu_C2_fourthPressureSet_i16_loop5_QR3
QR3 pos 0 (47.0,51.75): move took 1.4140503406524658s
QR3 pos 1 (47.0,53.0): move took 0.3399956226348877s
QR3 pos 2 (47.0,54.25): move took 0.35997819900512695s
QR3 pos 3 (47.0,55.5): move took 0.35997819900512695s
QR3 pos 4 (47.0,56.75): move took 0.3399810791015625s
QR3 pos 5 (48.25,51.75): move took 1.1499125957489014s
QR3 pos 6 (48.25,53.0): move took 0.3499777317047119s
QR3 pos 7 (48.25,54.25): move took 0.34999632835388184s
QR3 pos 8 (48.25,55.5): move took 0.3499791622161865s
QR3 pos 9 (48.25,56.75): move took 0.3499789237976074s
QR3 pos 10 (49.5,51.75): move took 1.1499300003051758s
QR3 pos 11 (49.5,53.0): move took 0.3399791717529297s
QR3 pos 12 (49.5,54.25): move took 0.3499612808227539s
QR3 pos 13 (49.5,55.5): move took 0.34998059272766113s
QR3 po

QR1 pos 6 (48.25,52.0): move took 0.349994421005249s
QR1 pos 7 (48.25,53.25): move took 0.3399968147277832s
QR1 pos 8 (48.25,54.5): move took 0.3499789237976074s
QR1 pos 9 (48.25,55.75): move took 0.3499948978424072s
QR1 pos 10 (49.5,50.75): move took 1.1399321556091309s
QR1 pos 11 (49.5,52.0): move took 0.34999561309814453s
QR1 pos 12 (49.5,53.25): move took 0.3499784469604492s
QR1 pos 13 (49.5,54.5): move took 0.3499791622161865s
QR1 pos 14 (49.5,55.75): move took 0.34999561309814453s
QR1 pos 15 (50.75,50.75): move took 1.139930248260498s
QR1 pos 16 (50.75,52.0): move took 0.349961519241333s
QR1 pos 17 (50.75,53.25): move took 0.3499789237976074s
QR1 pos 18 (50.75,54.5): move took 0.3499608039855957s
QR1 pos 19 (50.75,55.75): move took 0.3499796390533447s
QR1 pos 20 (52.0,50.75): move took 1.3499350547790527s
QR1 pos 21 (52.0,52.0): move took 0.3499784469604492s
QR1 pos 22 (52.0,53.25): move took 0.34997987747192383s
QR1 pos 23 (52.0,54.5): move took 0.3399631977081299s
QR1 pos 24 (5

QR3 pos 19 (50.75,56.75): move took 0.3499610424041748s
QR3 pos 20 (52.0,51.75): move took 1.1699459552764893s
QR3 pos 21 (52.0,53.0): move took 0.35996031761169434s
QR3 pos 22 (52.0,54.25): move took 0.3399631977081299s
QR3 pos 23 (52.0,55.5): move took 0.349977970123291s
QR3 pos 24 (52.0,56.75): move took 0.3399779796600342s
Creating new SPEC file: insitu_C2_fourthPressureSet_i32_loop10_air
air pos 0 (49,52.75): move took 1.1940133571624756s
Scan 10 finished in 363.80307149887085s
Creating new SPEC file: insitu_C2_fourthPressureSet_i33_loop11_QR1
QR1 pos 0 (47.0,50.75): move took 0.7973225116729736s
QR1 pos 1 (47.0,52.0): move took 0.3499789237976074s
QR1 pos 2 (47.0,53.25): move took 0.3399627208709717s
QR1 pos 3 (47.0,54.5): move took 0.34999513626098633s
QR1 pos 4 (47.0,55.75): move took 0.34999680519104004s
QR1 pos 5 (48.25,50.75): move took 1.164252758026123s
QR1 pos 6 (48.25,52.0): move took 0.3499622344970703s
QR1 pos 7 (48.25,53.25): move took 0.35022425651550293s
QR1 pos 8 (

...

QR3 pos 22 (52.0,54.25): move took 0.35997819900512695s
QR3 pos 23 (52.0,55.5): move took 0.3399772644042969s
QR3 pos 24 (52.0,56.75): move took 0.339979887008667s
Creating new SPEC file: insitu_C2_fourthPressureSet_i170_loop56_air
air pos 0 (49,52.75): move took 1.210474967956543s
Scan 56 finished in 318.40003204345703s
Creating new SPEC file: insitu_C2_fourthPressureSet_i171_loop57_QR1
QR1 pos 0 (47.0,50.75): move took 0.7887387275695801s
QR1 pos 1 (47.0,52.0): move took 0.3499622344970703s
QR1 pos 2 (47.0,53.25): move took 0.34997987747192383s
QR1 pos 3 (47.0,54.5): move took 0.3399965763092041s
QR1 pos 4 (47.0,55.75): move took 0.3499772548675537s
QR1 pos 5 (48.25,50.75): move took 1.1599128246307373s
QR1 pos 6 (48.25,52.0): move took 0.3499782085418701s
QR1 pos 7 (48.25,53.25): move took 0.369978666305542s
QR1 pos 8 (48.25,54.5): move took 0.339963436126709s
QR1 pos 9 (48.25,55.75): move took 0.3499617576599121s
QR1 pos 10 (49.5,50.75): move took 1.1599276065826416s
QR1 pos 11 (49

QR3 pos 6 (48.25,53.0): move took 0.3499774932861328s
QR3 pos 7 (48.25,54.25): move took 0.349961519241333s
QR3 pos 8 (48.25,55.5): move took 0.3499946594238281s
QR3 pos 9 (48.25,56.75): move took 0.3499774932861328s
QR3 pos 10 (49.5,51.75): move took 1.1599290370941162s
QR3 pos 11 (49.5,53.0): move took 0.3699758052825928s
QR3 pos 12 (49.5,54.25): move took 0.349977970123291s
QR3 pos 13 (49.5,55.5): move took 0.34999585151672363s
QR3 pos 14 (49.5,56.75): move took 0.3499624729156494s
QR3 pos 15 (50.75,51.75): move took 1.339935302734375s
QR3 pos 16 (50.75,53.0): move took 0.3399965763092041s
QR3 pos 17 (50.75,54.25): move took 0.3499794006347656s
QR3 pos 18 (50.75,55.5): move took 0.35997653007507324s
QR3 pos 19 (50.75,56.75): move took 0.3499619960784912s
QR3 pos 20 (52.0,51.75): move took 1.149949312210083s
QR3 pos 21 (52.0,53.0): move took 0.3499622344970703s
QR3 pos 22 (52.0,54.25): move took 0.339984655380249s
QR3 pos 23 (52.0,55.5): move took 0.34999537467956543s
QR3 pos 24 (52.

QR1 pos 16 (50.75,52.0): move took 0.34999632835388184s
QR1 pos 17 (50.75,53.25): move took 0.3499794006347656s
QR1 pos 18 (50.75,54.5): move took 0.3399629592895508s
QR1 pos 19 (50.75,55.75): move took 0.3499634265899658s
QR1 pos 20 (52.0,50.75): move took 1.3499186038970947s
QR1 pos 21 (52.0,52.0): move took 0.35997819900512695s
QR1 pos 22 (52.0,53.25): move took 0.34999537467956543s
QR1 pos 23 (52.0,54.5): move took 0.35997676849365234s
QR1 pos 24 (52.0,55.75): move took 0.3499791622161865s
Creating new SPEC file: insitu_C2_fourthPressureSet_i187_loop62_QR3
QR3 pos 0 (47.0,51.75): move took 1.3991551399230957s
QR3 pos 1 (47.0,53.0): move took 0.3399794101715088s
QR3 pos 2 (47.0,54.25): move took 0.3499784469604492s
QR3 pos 3 (47.0,55.5): move took 0.3699774742126465s
QR3 pos 4 (47.0,56.75): move took 0.3499782085418701s
QR3 pos 5 (48.25,51.75): move took 1.3499352931976318s
QR3 pos 6 (48.25,53.0): move took 0.3499786853790283s
QR3 pos 7 (48.25,54.25): move took 0.3499767780303955s
Q

QR1 pos 0 (47.0,50.75): move took 0.8002219200134277s
QR1 pos 1 (47.0,52.0): move took 0.34998059272766113s
QR1 pos 2 (47.0,53.25): move took 0.34999752044677734s
QR1 pos 3 (47.0,54.5): move took 0.3499603271484375s
QR1 pos 4 (47.0,55.75): move took 0.349994421005249s
QR1 pos 5 (48.25,50.75): move took 1.1499485969543457s
QR1 pos 6 (48.25,52.0): move took 0.35999464988708496s
QR1 pos 7 (48.25,53.25): move took 0.3499786853790283s
QR1 pos 8 (48.25,54.5): move took 0.3699941635131836s
QR1 pos 9 (48.25,55.75): move took 0.3399982452392578s
QR1 pos 10 (49.5,50.75): move took 1.349919080734253s
QR1 pos 11 (49.5,52.0): move took 0.3499782085418701s
QR1 pos 12 (49.5,53.25): move took 0.35997772216796875s
QR1 pos 13 (49.5,54.5): move took 0.35999512672424316s
QR1 pos 14 (49.5,55.75): move took 0.3499608039855957s
QR1 pos 15 (50.75,50.75): move took 1.339918851852417s
QR1 pos 16 (50.75,52.0): move took 0.35996174812316895s
QR1 pos 17 (50.75,53.25): move took 0.3399813175201416s
QR1 pos 18 (50.7

KeyboardInterrupt: 

Release control of SPEC

In [ ]:
r = requests.get(bServer + "release_remote_control")
print(r.json())